Python environment

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
sc.settings.n_jobs = 12

In [ ]:
sc.set_figure_params(dpi=100, dpi_save = 300, color_map = 'viridis')
sc.settings.verbosity = 1
sc.logging.print_header()
plt.rcParams['pdf.fonttype'] = 42
sc.settings.figdir = 'figures'

scanpy==1.8.2 anndata==0.7.5 umap==0.5.2 numpy==1.20.2 scipy==1.5.2 pandas==1.1.2 scikit-learn==0.23.2 statsmodels==0.12.0 python-igraph==0.9.1 pynndescent==0.5.6


In [ ]:
import matplotlib as mpl
# https://github.com/theislab/scanpy/issues/1720
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

### Read pre-processed data

In [ ]:
adata = sc.read('data/core_GBmap.h5ad')
adata

AnnData object with n_obs × n_vars = 431535 × 36498
    obs: 'author', 'patient', 'celltype_malignant', 'celltype_major', 'celltype_minor', 'gender', 'location', 'EGFR', 'platform', 'method', 'age', 'MET', 'p53', 'TERT', 'ATRX', 'PTEN', 'MGMT', 'sample', 'celltype_original', 'KI_67', 'region', 'Tissue', 'chr1p19q', 'stage', 'Verhaak_classification', 'PDGFR', 'gs_prediction', 'gs_prediction_main', 'gs_prediction_detailed', 'cnv_full_geneset', 'cnv_filtered_geneset', 'celltype_assigned'
    var: 'gene_ids', 'feature_types', 'genome'

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.raw = adata # keep full dimension safe
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=5000,
    batch_key="author",
    layer="counts",
    subset=True)

In [ ]:
### train the model including only samples that have used cells (not nuclei)
source_adata = adata[adata.obs['method'].isin(['cell'])].copy()

In [ ]:
source_adata.layers["counts"] = source_adata.X.copy()

In [ ]:
metadata = pd.read_csv('data/final_annotation.csv', index_col=0)
metadata

,annotation_level_3,annotation_level_4,annotation_level_2,annotation_level_1
PJ017_0,MES-like,MES-like hypoxia independent,Differentiated-like,Neoplastic
PJ017_1,OPC-like,OPC-like,Stem-like,Neoplastic
PJ017_2,TAM-BDM,TAM-BDM hypoxia/MES,Myeloid,Non-neoplastic
PJ017_3,MES-like,MES-like hypoxia independent,Differentiated-like,Neoplastic
PJ017_4,AC-like,AC-like,Differentiated-like,Neoplastic
...,...,...,...,...
E100_GEX_TTTGTCACATGCAATC-1,CD4/CD8,Stress sig,Lymphoid,Non-neoplastic
E100_GEX_TTTGTCAGTAGAGCTG-1,CD4/CD8,CD8 EM,Lymphoid,Non-neoplastic
E100_GEX_TTTGTCAGTATCGCAT-1,CD4/CD8,Stress sig,Lymphoid,Non-neoplastic
E100_GEX_TTTGTCATCTGTCTCG-1,CD4/CD8,CD8 EM,Lymphoid,Non-neoplastic


In [ ]:
source_adata.obs['CellID'] = metadata['annotation_level_3'].values

In [ ]:
sc.pp.normalize_total(source_adata)
sc.pp.log1p(source_adata)
source_adata.raw = source_adata # keep full dimension safe
sc.pp.highly_variable_genes(
    source_adata,
    n_top_genes=5000,
    batch_key="author",
    layer="counts",
    subset=True)

... storing 'author' as categorical
... storing 'patient' as categorical
... storing 'method' as categorical
... storing 'CellID' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical
/hpc/pmc_stunnenberg/cruiz/miniconda3/envs/scarches/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Set relevant anndata.obs labels and training length

In [ ]:
condition_key = 'author'
cell_type_key = 'CellID'

vae_epochs = 500
scanvi_epochs = 200
surgery_epochs = 500

early_stopping_kwargs = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "patience": 10,
    "threshold": 0,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}
early_stopping_kwargs_scanvi = {
    "early_stopping_metric": "accuracy",
    "save_best_state_metric": "accuracy",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}
early_stopping_kwargs_surgery = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}

### Create SCANVI model and train it on fully labelled reference dataset

In [ ]:
sca.dataset.setup_anndata(source_adata, layer = 'counts', batch_key=condition_key, labels_key=cell_type_key)

INFO     Using batches from adata.obs["author"]                                              
INFO     Using labels from adata.obs["CellID"]                                               
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 338564 cells, 5000 vars, 17       
         batches, 20 labels, and 0 proteins. Also registered 0 extra categorical covariates  
         and 0 extra continuous covariates.                                                  
INFO     Please do not further modify adata until model is trained.                          


In [ ]:
vae = sca.models.SCANVI(
    source_adata,
    "Unknown",
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
    use_cuda = True # indicate to use gpu!
)

In [ ]:
print("Labelled Indices: ", len(vae._labeled_indices))
print("Unlabelled Indices: ", len(vae._unlabeled_indices))

Labelled Indices:  338564
Unlabelled Indices:  0


In [ ]:
vae.train(
    n_epochs_unsupervised=vae_epochs,
    n_epochs_semisupervised=scanvi_epochs,
    unsupervised_trainer_kwargs=dict(early_stopping_kwargs=early_stopping_kwargs),
    semisupervised_trainer_kwargs=dict(metrics_to_monitor=["elbo", "accuracy"],
                                       early_stopping_kwargs=early_stopping_kwargs_scanvi),
    frequency=1
)

INFO     Training Unsupervised Trainer for 500 epochs.                                       
INFO     Training SemiSupervised Trainer for 200 epochs.                                     
INFO     KL warmup for 400 epochs                                                            
Training...:  69%|██████▊   | 343/500 [3:18:49<1:37:06, 37.11s/it]INFO                                                                                         
         Stopping early: no improvement of more than 0 nats in 10 epochs                     
INFO     If the early stopping criterion is too strong, please instantiate it with different 
         parameters in the train method.                                                     
Training...:  69%|██████▊   | 343/500 [3:19:30<1:31:19, 34.90s/it]
INFO     Training is still in warming up phase. If your applications rely on the posterior   
         quality, consider training for more epochs or reducing the kl warmup.               
INFO     Training tim

In [ ]:
ref_path = 'scarches_models/zenodo_model/'
vae.save(ref_path, overwrite=True)

In [ ]:
source_adata.write_h5ad('scarches_models/zenodo_model/scarches_GBmap_core.h5ad')

... storing 'feature_types' as categorical
... storing 'genome' as categorical
